In [ ]:
import pandas as pd
import numpy as np
import pywhatkit as wpp
from datetime import datetime, timedelta

#SetOptions para ver los dataframes
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',False)

In [ ]:
def dias_alertas():
    hoy = datetime.now()
    dias_semana = {'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3, 'viernes': 4, 'sábado': 5, 'domingo': 6}

    # Encuentra el último martes y jueves
    last_tuesday = hoy - timedelta(days=(hoy.weekday() - dias_semana['martes']) % 7)
    last_thursday = hoy - timedelta(days=(hoy.weekday() - dias_semana['jueves']) % 7)

    # Devuelve la fecha más reciente entre el último martes y jueves
    return max(last_tuesday, last_thursday).strftime('%d')

# Llama a la función y muestra el resultado
# print(dias_alertas())

In [ ]:
path = 'Data/'

#df_matries = pd.read_excel(path + 'Román - M{:d} Seguimiento.xlsx', sheet_name='Estatus').format(mes)
database = pd.read_excel(path + 'BASE CNCI.xlsx')
inactivos = pd.read_excel(path + 'Alertas tempranas {:2} de Abril 2024.xlsx'.format(dias_alertas()), sheet_name='Inactivo')

#### Limpieza del Database

Dropear columnas innecesarias

In [ ]:
#Renombrar el nombre de la columna NOMBRE
database.rename(columns = {'NOMBRE                                                                                                                                      ' : "Nombre", 'MATRICULA' : "Matricula"}, inplace= True)

database = database[['Nombre', 'Matricula', 'CE CANALIZADO', 'CARRERA', 'M4', 'Correo', 'Teléfono', 'Teléfono2', 'Teléfono3', 'CURP', 'Agente ', 'Asesor Académico', 'Asesor Financiero ', 'Asesor Escolar']]

# Filtro Cadereyta 
database = database[database['CE CANALIZADO'] == 'CE CADEREYTA']

#### Limpieza y Filtro de Inactivos

In [ ]:
inactivos = inactivos.drop(columns=['Registro 1', 'Registro 2', 'Registro 3', 'Registro 4', 'Registro 5', 'Subproyecto'])
inactivos = inactivos[inactivos['Centro Enlace'].isin(['CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA'])]

In [ ]:
# Juntas las materias
inactivos_agrup = inactivos.groupby("Matricula")["Grupo"].agg(lambda x: ", ".join(x)).reset_index().rename(columns={'Grupo':'Materias'})
inactivos = pd.merge(inactivos, inactivos_agrup, on = "Matricula", how = "left")
inactivos.shape # Materias agrupadas en la última columna

In [ ]:
# Reacomodo de las materias
columna_extra = inactivos["Materias"]
inactivos.drop(columns = ["Materias", 'Modelo'], inplace=True)
inactivos["Grupo"] = columna_extra
inactivos = inactivos.drop_duplicates(subset='Nombre')
inactivos.shape # Sin duplicados

In [ ]:
imagen = 'img/Cerro.jpeg'
path2 = 'infodata/'

try:
    matric_error = pd.read_csv(path2 + 'matriculas_error.csv')
    matric_yacontactadas = pd.read_csv(path2 + 'matriculas_yacontactadas.csv') 
    
    matric_error = matric_error.dropna().values.flatten().tolist()
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist()
except FileNotFoundError:
    matric_error = pd.DataFrame()
    matric_yacontactadas = pd.DataFrame()

In [ ]:
# Merge nuestros df, con el fin de no llamar a dos df en el for.
inactivos = pd.merge(inactivos, database[['Matricula','Teléfono', 'Teléfono2', 'M4']], on = 'Matricula')

#### Ciclo de mensajes

In [ ]:
for index, row in inactivos.iterrows():
    alumno = row['Nombre']
    matricula = row['Matricula']
    materias = row['Grupo']
    status = row['Estatus']
    telefono = row['Teléfono']
    if not np.isnan(telefono):
        telefono = '+52 ' + str(telefono)
    if matricula in matric_error:
        if not np.isnan(row['Teléfono2']):
            telefono2 = '+52 ' + str(row['Teléfono2'])
            telefono = telefono2
            print('Matricula con error: {}, Alumno: {}'.format(matricula, alumno))
    
    if status == ('Inactivo'):
        mencion_stat = 'veo que aún no has ingresado a tus materias: *' + materias +'*'
    elif status == ('Ausente'):
        mencion_stat = 'veo que ya has ingresado a tus materias: *'+ materias +'* \n ¿Has logrado avanzar con tus tareas?' 
        
    if matricula in matric_yacontactadas:
        print('Matricula antes contactada: {}, Alumno: {} '.format(matricula, alumno))    
    else:
        mensaje = 'Hola, que tal ' + alumno + ' ' + matricula + ', soy _Román Rodríguez_ Tutor Presencial del CNCI, me comunico contigo ya que ' + mencion_stat + ' ¿Hay algo en lo que pueda apoyarte?'
        print('Enviado a: ', matricula, alumno)
        #wpp.sendwhats_image(telefono, imagen, mensaje, 25, True)
        # print(telefono, matricula, alumno)
